<a href="https://colab.research.google.com/github/RodriMenendez/BIO_LM_SURF/blob/main/practice/Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import numpy as np

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [16]:
path = '/content/drive/MyDrive/SURF_2023/Practice'

transform = transforms.Compose([transforms.ToTensor()])
train_data = torchvision.datasets.CIFAR10(root=path+'/cifar10', train=True, transform=transform, download=True)

Files already downloaded and verified


In [9]:
test_data = torchvision.datasets.CIFAR10(root=path+'/cifar10', train=False, download=False)